In [1]:
%matplotlib qt
import os.path as op
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reading data 

In [3]:
#Setup for reading the raw data
subj = "sub-009"
examples_dir = "E:\\Amin\\PDM\\Data\\eeg_data\\" + subj + "\\sourcedata-eeg_outside-MRT\\eeg"
vhdr_file = op.join(examples_dir, subj + "_task-pdm_acq-outsideMRT_eeg.vhdr")
raw = mne.io.read_raw_brainvision(vhdr_file, misc='auto')
# preload=True
#loading data 
raw.load_data()

Extracting parameters from E:\Amin\PDM\Data\eeg_data\sub-009\sourcedata-eeg_outside-MRT\eeg\sub-009_task-pdm_acq-outsideMRT_eeg.vhdr...
Setting channel info structure...


<ipython-input-3-e99dce2b49b9>:5: RuntimeWarning: No coordinate information found for channels ['EOG', 'ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  raw = mne.io.read_raw_brainvision(vhdr_file, misc='auto')


Reading 0 ... 6389699  =      0.000 ...  1277.940 secs...


<RawBrainVision | sub-009_task-pdm_acq-outsideMRT_eeg.eeg, 64 x 6389700 (1277.9 s), ~3.05 GB, data loaded>

In [5]:
#set channel types explicitly.
raw.set_channel_types({'EOG':'eog'});
raw.set_channel_types({'ECG':'ecg'});

<ipython-input-5-dfac93531578>:2: RuntimeWarning: The unit for channel(s) EOG has changed from NA to V.
  raw.set_channel_types({'EOG':'eog'});
<ipython-input-5-dfac93531578>:3: RuntimeWarning: The unit for channel(s) ECG has changed from NA to V.
  raw.set_channel_types({'ECG':'ecg'});


# Preaprocessing

### 1. Resampling

In [6]:
raw.resample(1024, npad="auto");

### 2. Filtering the data

In [7]:
#A high-pass filter with 1 Hz cutoff frequency for removing low-frequency drifts and before ICA 
raw.filter(l_freq = 1, h_freq=10, method='iir', picks=['eeg'])

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 25.00 Hz: -6.02, -6.02 dB



<RawBrainVision | sub-009_task-pdm_acq-outsideMRT_eeg.eeg, 64 x 1308611 (1277.9 s), ~639.1 MB, data loaded>

In [168]:
_=raw.plot_psd(fmin=0, fmax=30)

Effective window size : 2.000 (s)


### 3. Rereference

In [8]:
#If projection=True, the average reference is added as a projection and is not applied to the data
#It can be applied afterwards with the apply_proj method
raw.set_eeg_reference('average', projection=True).apply_proj();

Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


### 4. Visual Inspection

###### 4.1 Plot continuous data

In [9]:
_=raw.plot()

### 5. ICA

###### 5.1 EOG and ECG

In [11]:
#A summary of how the ocular artifact manifests across each channel
eog_evoked = mne.preprocessing.create_eog_epochs(raw).average()
eog_evoked.apply_baseline(baseline=(None, -0.2))
_=eog_evoked.plot_joint()

Using EOG channel: EOG
EOG channel index for this subject is: [30]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10240 samples (10.000 sec)

Now detecting blinks and generating corresponding events
Found 264 significant peaks
Number of EOG events detected: 264
Not setting metadata
Not setting metadata
264 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 264 events and 1025 original time points ...
1 bad 

In [12]:
#A summary of how the the heartbeat artifacts manifests across each channel
ecg_evoked = mne.preprocessing.create_ecg_epochs(raw).average()
ecg_evoked.apply_baseline(baseline=(None, -0.2))
_=ecg_evoked.plot_joint()

Using channel ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 10240 samples (10.000 sec)

Number of ECG events detected : 1488 (average pulse 69 / min.)
Not setting metadata
Not setting metadata
1488 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1488 events and 1025 original time points ...
1 bad epochs dropped
Applying baseline correction (mode: mean)
Projections have already been applied. Setting proj attribute to True.


###### 5.2 run ICA

In [13]:
ica = mne.preprocessing.ICA(n_components=60, random_state=97, method='fastica')
ica.fit(raw)

Fitting ICA to data using 62 channels (please be patient, this may take a while)


<ipython-input-13-0a02b06026fa>:1: DeprecationWarning: Version 0.23 introduced max_iter="auto", setting max_iter=1000 for `fastica` and max_iter=500 for `infomax` and `picard`. The current default of max_iter=200 will be changed to "auto" in version 0.24.
  ica = mne.preprocessing.ICA(n_components=60, random_state=97, method='fastica')


    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 60 components
    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 99.4s.


<ICA | raw data decomposition, fit (fastica): 1308611 samples, 60 components, channels used: "eeg">

In [14]:
ica.plot_components()

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>]

##### 5.3 EOG component rejection

In [15]:
#Using an EOG channel to select ICA components
ica.exclude = []
# find which ICs match the EOG pattern
eog_indices, eog_scores = ica.find_bads_eog(raw)
ica.exclude = eog_indices

# barplot of ICA component "EOG match" scores
ica.plot_scores(eog_scores)


if len(eog_indices)>0:
    # plot diagnostics
    ica.plot_properties(raw, picks=eog_indices)

    # plot ICs applied to raw data, with EOG matches highlighted
    #ica.plot_sources(raw)

    # plot ICs applied to the averaged EOG epochs, with EOG matches highlighted
    #ica.plot_sources(eog_evoked)

Using EOG channel: EOG
    Applying projection operator with 1 vector (pre-whitener application)
... filtering ICA sources
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 10240 samples (10.000 sec)

... filtering target
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz

###### 5.4 ECG component rejection

In [16]:
ica.exclude = []
#
## find which ICs match the ECG pattern
ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method='ctps')
ica.exclude = ecg_indices
#
# barplot of ICA component "ECG match" scores
ica.plot_scores(ecg_scores)

if len(ecg_indices)>0:
    # plot diagnostics
    ica.plot_properties(raw, picks=ecg_indices)

    # plot ICs applied to raw data, with ECG matches highlighted
    #ica.plot_sources(raw)

    # plot ICs applied to the averaged ECG epochs, with ECG matches highlighted
    #ica.plot_sources(ecg_evoked)

Using threshold: 0.16 for CTPS ECG detection
Using channel ECG to identify heart beats.
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 10240 samples (10.000 sec)

Number of ECG events detected : 1488 (average pulse 69 / min.)
Not setting metadata
Not setting metadata
1488 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 1488 events and 1025 original time points ...
1 bad epochs dropped
    Applying projection operator with 1 vector (pre-whitener application)
    Applying projection o

###### 5.5 select IC artifact manually

In [44]:
# Topography
ica.plot_properties(raw, picks=59)

    Applying projection operator with 1 vector (pre-whitener application)
    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
Not setting metadata
638 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped


[<Figure size 700x600 with 6 Axes>]

In [46]:
ica.exclude = [0,29,1,24,40,47,48,50,57]

In [47]:
ica.apply(raw)

Applying ICA to Raw instance
    Applying projection operator with 1 vector (pre-whitener application)
    Transforming to ICA space (60 components)
    Zeroing out 9 ICA components
    Projecting back using 62 PCA components


<RawBrainVision | sub-009_task-pdm_acq-outsideMRT_eeg.eeg, 64 x 1308611 (1277.9 s), ~639.1 MB, data loaded>

In [153]:
#epochs.plot(picks=['F1'])
_=raw.plot()

### 6. Segmenting continuous data into epochs and setting Ampilitude ceriteria 

##### 6.1 epoching 

In [48]:
#print(set(raw.annotations.description))
events_from_annot, event_dict = mne.events_from_annotations(raw)
event_dict = {'10':10, '11':11, '20':20, '21':21,
              '30':30, '31':31, '40':40, '41':41}
#ceriteria Ampilitude
#reject_criteria = dict(eeg=100e-6) # 100 μV     
epochs = mne.Epochs(raw, events_from_annot, event_id=event_dict, tmin=-0.100, tmax=.400,
                    baseline = (None,0), preload=True, picks=['eeg'])

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  5', 'Stimulus/S  6', 'Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41', 'Stimulus/S 73', 'Stimulus/S 74', 'Stimulus/S 75', 'Stimulus/S 76', 'SyncStatus/Sync Off']
Not setting metadata
Not setting metadata
288 matching events found
Setting baseline interval to [-0.25, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Loading data for 288 events and 1537 original time points ...
0 bad epochs dropped


In [49]:
epochs

Number of events,288
Events,10: 3611: 3620: 3621: 3630: 3631: 3640: 3641: 36
Time range,-0.250 – 1.250 sec
Baseline,-0.250 – 0.000 sec


###### 6.1 equalize event

In [50]:
#epochs.equalize_event_counts(event_dict)

# 7. Save final preprocessing file

In [ ]:
epochs.save(examples_dir + "\\"+ subj+"_final_preprocessing_IIR_60_25hz.fif", overwrite=True)